### Data filtering - V 20220520

This notebook filters H&H.

Filter by number transactions per customers, number minimum number of article instances and max number of customers

Ouptut is
- transactions_ddup_datef_artif_custf.csv 
- transactions_ddup_datef_artif_custf_custr.csv (same as previous but with a max number of customers)

Asumes data in data directory
    

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
articles = pd.read_csv("./data/articles.csv")
customers = pd.read_csv("./data/customers.csv")
transactions = pd.read_csv("./data/transactions_train.csv")

Displays basic data

In [3]:
print ("Number articles -> ", len(articles))
print ("Number customers -> ", len(customers))
print ("Number transaction -> ", len(transactions))

Number articles ->  105542
Number customers ->  1371980
Number transaction ->  31788324


In [4]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


Before any filter remove duplicate records (if an item is bought multiples times in a purchase, records are duplicate)

In [5]:
transactions_ddup=transactions.drop_duplicates()

In [6]:
print ("Duplicate records removed -> " ,len(transactions)-len(transactions_ddup))

Duplicate records removed ->  2974905


### **Time span**

Temporal range

In [7]:
transactions_ddup.agg({
    "t_dat" :["min","max"]
})

,t_dat
min,2018-09-20
max,2020-09-22


## Data filtering

In the following variables we specify
- `ini_data` (initial data in format aaaa-mm-dd)
- `min_trans_per_customer` (if a customer has less than this number of transactions, its transactions will be removed)
- `min_trans_per_article` (if an article has less than this number of transactions, its transactions will be removed)


In [8]:
ini_data= "2019-09-22"
min_trans_per_article=5
min_trans_per_customer=20


### Date filtering

In [9]:
print ("Initial data -> ", ini_data)
df_mask=transactions_ddup["t_dat"]>=ini_data # mask
transactions_ddup_datef=transactions_ddup[df_mask] # apply mask
print ("Number transactions -> ", len(transactions_ddup_datef))
transactions_ddup_datef.head()

Initial data ->  2019-09-22
Number transactions ->  13564075


,t_dat,customer_id,article_id,price,sales_channel_id
16855951,2019-09-22,000fb6e772c5d0023892065e659963da90b1866035558e...,695632095,0.033881,2
16855952,2019-09-22,0014b19aa47d031d859ae8d8f776524cdd35349db69d64...,787648001,0.016932,2
16855953,2019-09-22,0023f9e96ae935c11bc2c950f317b73facea404dde95a3...,767496001,0.015237,1
16855954,2019-09-22,0023f9e96ae935c11bc2c950f317b73facea404dde95a3...,766826002,0.050831,1
16855955,2019-09-22,0023f9e96ae935c11bc2c950f317b73facea404dde95a3...,655710010,0.050831,1


### Article and customer filtering

Filtering is based in the number a customer or article comes up in 'customer_id' and 'article_id'. We create a panda serie with the number transactions of identical customers/articles. Then we use the panda serie to select the records in the unfiltered version of the file.

Ref -> https://stackoverflow.com/questions/48628417/how-to-select-rows-in-pandas-dataframe-where-value-appears-more-than-once


**Article filtering**

Starting poing is `transactions_ddup_datef` (deduplicate and with data filter)

In [10]:
v_art=transactions_ddup_datef.article_id.value_counts() # counts number records for each article_id

In [11]:
print ("Article data ")
v_art

Article data 


706016001    25579
720125001    15608
706016002    15062
372860001    14201
759871002    13926
             ...  
717664004        1
685471001        1
651330003        1
617465001        1
533261032        1
Name: article_id, Length: 70976, dtype: int64

In [12]:
v_art.describe()

count    70976.000000
mean       191.107910
std        501.433742
min          1.000000
25%          5.000000
50%         27.000000
75%        160.000000
max      25579.000000
Name: article_id, dtype: float64

In [13]:
# select articles where v_art.gt(min_trans_per_article) is true
transactions_ddup_datef_artif= \
   transactions_ddup_datef[transactions_ddup_datef.article_id.isin(v_art.index[v_art.gt(min_trans_per_article-1)])]

In [14]:
# debug 
#
# transactions_ddup_datef_artif.describe()

In [15]:
# debug 
#
# apply filter to verify that indeed the min number of transactions x customer is correct
# v_debug=transactions_ddup_datef_artif.article_id.value_counts() # counts number records for each article_id
#v_debug.describe()

**Customer filtering**

Starting poing is `transactions_ddup_datef_datef` (deduplicate and with data filter and article filter)

In [16]:
v_cust=transactions_ddup_datef_artif.customer_id.value_counts() # counts number records for each customer_id

In [17]:
print ("Customer data ")
v_cust

Customer data 


be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee985513d9e8e53c6d91b    862
a65f77281a528bf5c1e9f270141d601d116e1df33bf9df512f495ee06647a9cc    742
b637a3e7d8b0caa947aaefd609b8d84a9ee962cf0a52a51bac507ffc2bf1b741    728
b4db5e5259234574edfff958e170fe3a5e13b6f146752ca066abca3c156acc71    727
55d15396193dfd45836af3a6269a079efea339e875eff42cc0c228b002548a9d    627
                                                                   ... 
2880444cb4592993c95e87c176676981ae9aad51124a494724cbf9a65e370fdb      1
29a91769e5d3cbd0ef19a16d93760d8dfce46a4ba96d81acedb8fffb88321c6a      1
29ae0df422f197bd2bdd50e262541a20da5afeb70f731f28c5aba9a9ca82da5d      1
29c37662687306a9b86316f41eded48a1c2dc5e20319a0d38a884b930982eb29      1
ff6f55a51af284b71dcd264396b299e548f968c1769e71148763bd4fce26b40d      1
Name: customer_id, Length: 994820, dtype: int64

In [18]:
v_cust.describe()

count    994820.000000
mean         13.600082
std          19.343804
min           1.000000
25%           3.000000
50%           7.000000
75%          17.000000
max         862.000000
Name: customer_id, dtype: float64

In [19]:
# select articles where v_cust.gt(min_trans_per_customer) is true
transactions_ddup_datef_artif_custf= \
   transactions_ddup_datef_artif[transactions_ddup_datef_artif.customer_id.isin(v_cust.index[v_cust.gt(min_trans_per_customer-1)])]

In [20]:
# debug 
#
#transactions_ddup_datef_artif_custf.describe()


In [21]:
# debug 
#
# apply filter to verify that indeed the min number of transactions x customer is correct
#v_debug=transactions_ddup_datef_artif_custf.customer_id.value_counts() # counts number records for each customer_id
#v_debug.describe()

In [22]:
transactions_ddup_datef_artif_custf.describe()

,article_id,price,sales_channel_id
count,8.470046e+06,8.470046e+06,8.470046e+06
mean,7.590754e+08,2.829068e-02,1.735564e+00
std,1.253690e+08,1.908699e-02,4.410324e-01
min,1.087750e+08,3.389831e-05,1.000000e+00
25%,7.153430e+08,1.661017e-02,1.000000e+00
50%,7.924690e+08,2.540678e-02,2.000000e+00
75%,8.412280e+08,3.388136e-02,2.000000e+00
max,9.562170e+08,5.067797e-01,2.000000e+00


**Custormer reduction**

As the numer of records is too large we will use a subset with less records


In [23]:
print ("Current number of transactions -", len(transactions_ddup_datef_artif_custf) )

Current number of transactions - 8470046


Auxiliar dataserie with customer data

In [24]:
v_aux=transactions_ddup_datef_artif_custf.customer_id.value_counts() 

In [25]:
v_aux.describe()

count    207491.000000
mean         40.821269
std          27.631893
min          20.000000
25%          24.000000
50%          32.000000
75%          47.000000
max         862.000000
Name: customer_id, dtype: float64

In [26]:
v_aux

be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee985513d9e8e53c6d91b    862
a65f77281a528bf5c1e9f270141d601d116e1df33bf9df512f495ee06647a9cc    742
b637a3e7d8b0caa947aaefd609b8d84a9ee962cf0a52a51bac507ffc2bf1b741    728
b4db5e5259234574edfff958e170fe3a5e13b6f146752ca066abca3c156acc71    727
55d15396193dfd45836af3a6269a079efea339e875eff42cc0c228b002548a9d    627
                                                                   ... 
571bad7b761931d65a741826735c94d533b39fa71c7ba70fcc9b0253f87c2003     20
7a914aebab66fc30cb0288615558b9bb8e878239f278aa81e327090efe09b0ae     20
7a9310866dcc7d3461857cc903532708337c49e28aa7d9d109508ed3a9e9fb04     20
7a9d1b6641a3b7a35fb2cbb9c908d579ac5a552e0f81b6b949770645caedfa67     20
c878c66b040e01adfec5a334a7afde32913eff32db7b83b6ed97db22e7ff05f6     20
Name: customer_id, Length: 207491, dtype: int64

We will suffle data

In [27]:
from sklearn.utils import shuffle
v_aux= shuffle(v_aux)
print ("Number of customers -> ", len(v_aux))

Number of customers ->  207491


We set here the number total cust

In [28]:
tot_cust=10000
v_aux=v_aux[0:tot_cust]

In [29]:
print ("Number of selected random customers -> ", len (v_aux))

Number of selected random customers ->  10000


In [30]:
# debug   v_aux.index is the list of selected customers
# v_aux.index

In [31]:
# custr -> customer reduction we filter transactions_ddup_datef_artif_custf
# with v_aux.index to get our transactions_ddup_datef_artif_custf_custr
transactions_ddup_datef_artif_custf_custr= \
   transactions_ddup_datef_artif_custf[ \
        transactions_ddup_datef_artif_custf.customer_id.isin \
        (v_aux.index)]

In [32]:
print ("Transactions with data filter, customer filter and customer red ->" , \
      len(transactions_ddup_datef_artif_custf_custr))

Transactions with data filter, customer filter and customer red -> 407717


In [33]:
transactions_ddup_datef_artif_custf_custr.describe()

,article_id,price,sales_channel_id
count,4.077170e+05,407717.000000,407717.000000
mean,7.589289e+08,0.028292,1.740435
std,1.250383e+08,0.019092,0.438396
min,1.087750e+08,0.000288,1.000000
25%,7.152310e+08,0.016661,1.000000
50%,7.923010e+08,0.025407,2.000000
75%,8.406040e+08,0.033881,2.000000
max,9.529380e+08,0.506780,2.000000


In [34]:
print ("File name for FULL filtered dataset:")
file_name="./data/transactions_ddup_{0}_nart_{1}_ncust_{2}.csv".format( \
  ini_data, min_trans_per_article, min_trans_per_customer)
print (file_name)
print ("File name for reduced filtered dataset (later defined):")
file_name_red="./data/transactions_ddup_{0}_nart_{1}_ncust_{2}_ncustr_{3}.csv".format( \
   ini_data, min_trans_per_article, min_trans_per_customer, tot_cust)
print (file_name_red)

File name for FULL filtered dataset:
./data/transactions_ddup_2019-09-22_nart_5_ncust_20.csv
File name for reduced filtered dataset (later defined):
./data/transactions_ddup_2019-09-22_nart_5_ncust_20_ncustr_10000.csv


In [35]:
#ini_data= "2019-09-22"
#min_trans_per_article=1000
#min_trans_per_customer=20

print ("Filter values:")
print ("Initial data -> ", ini_data)
print ("Minimum transactions per article -> ", min_trans_per_article)
print ("Minimum transactions per customer -> ", min_trans_per_customer)
print ("Customer reduction to customer -> ", tot_cust)
print ("Filtered FULL dataframe will be exported as csv as    -> ", file_name)
transactions_ddup_datef_artif_custf.to_csv(file_name)
print ("Filtered reduced dataframe will be exported as csv as -> ", file_name_red)
transactions_ddup_datef_artif_custf_custr.to_csv(file_name_red)
print ("Done!")


Filter values:
Initial data ->  2019-09-22
Minimum transactions per article ->  5
Minimum transactions per customer ->  20
Customer reduction to customer ->  10000
Filtered FULL dataframe will be exported as csv as    ->  ./data/transactions_ddup_2019-09-22_nart_5_ncust_20.csv
Filtered reduced dataframe will be exported as csv as ->  ./data/transactions_ddup_2019-09-22_nart_5_ncust_20_ncustr_10000.csv
Done!


In [ ]:
# debuf final verification for customers
#transactions_ddup_datef_artif_custf.customer_id.value_counts()

In [ ]:
# debuf final verification for article
#transactions_ddup_datef_artif_custf.article_id.value_counts()